# Main script 

This script runs the main simulation.

### Import packages
Run the following cell to import the necessary packages

In [ ]:
import scipy.ndimage
import projectcode as proj
import plotcode as plot
import numpy as np
import analysis_utils as au
import pipeline_helpers as ph
import matplotlib.pyplot as plt
from pathlib import Path
import scipy
from PIL import Image
from scipy.ndimage import gaussian_filter
from tqdm import tqdm
import seaborn as sns 
import image_processing as ip
from dataclasses import replace
import cProfile
import pstats
import poly_fit_analysis as pfa

### Define Parameters
Here you can define the parameters of the simulation. Tuning F and k will 
result in different simulation patterns emerging. For example:

Dots (lambda pattern in Pearson)

F = 0.037 

k = 0.065 


Stripes (kappa pattern in Pearson) 

F = 0.048 

k = 0.063 


In [2]:
params = proj.ModelParams(
    Du=2e-5, 
    Dv=1e-5, 
    F=0.048,
    k=0.063, # F0.037, k0.065 - dots, lambda, F0.048, k0.063 stripes, kappa
    dt=1,
    steps=35000,
    N=256
    )

### RUN SIMULATION
Run the cell below to run the simulation. 

    Returns:
    U_snap : np.ndarray
            Storage array of snapshots of u.
    V_snap : np.ndarray
        Storage array of snapshots of u. 
    u : np.ndarray, shape (N,N)
        Final result of u.
    v : np.ndarray, shape (N,N)
        Final result of v.

If save = True, saves U_snap, V_snap and relevant run info in the designated 
base directory, in a runs folder. The run is given the following ID:

    run_id = (
            f"{time_stamp}"
            + f"_F{params.F}"
            + f"_k{params.k}"
            + f"_S{steps}" 
            + f"_SI{save_interval}"
        )

This also appends a line to a csv master log file, with the following info:

    ["timestamp","Du","Dv","F","k","dt","dx","steps","path"]


In [ ]:
usnap, vsnap, u, v = \
      proj.simulation(p=params, base_dir = "data/simulations", save=False) 

print("successful run")


### Run simulation with profiler
This runs the simulation together with a profiler, which gives information on
how efficient the code is.

In [ ]:

profiler = cProfile.Profile()
profiler.enable()

usnap, vsnap, u, v  = proj.simulation(p=params, base_dir = "data/simulations", save=False) 

profiler.disable()

stats = pstats.Stats(profiler)
stats.sort_stats("cumulative").print_stats(20)  # Top 20 functions by cumulative time


100%|██████████| 35000/35000 [00:30<00:00, 1131.19it/s]


         2883783 function calls (2874468 primitive calls) in 30.958 seconds

   Ordered by: cumulative time
   List reduced from 367 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000   24.097   12.049 /Users/sophiawellman/Git/Project3_cnygren_sawellman/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3636(run_code)
    70000    0.027    0.000   21.046    0.000 /Users/sophiawellman/Git/Project3_cnygren_sawellman/.venv/lib/python3.13/site-packages/scipy/ndimage/_filters.py:949(convolve)
    70000    0.303    0.000   21.019    0.000 /Users/sophiawellman/Git/Project3_cnygren_sawellman/.venv/lib/python3.13/site-packages/scipy/ndimage/_filters.py:820(_correlate_or_convolve)
    70000   19.983    0.000   19.983    0.000 {built-in method scipy.ndimage._nd_image.correlate}
    70000    0.066    0.000    0.499    0.000 /Users/sophiawellman/Git/Project3_cnygren_sawellman/.venv/lib/python3.13/site-p

### Animation Plots
This creates animated plots for the simulation, saved under the designated file
name. 

In [ ]:
clipped_arr = plot.pearson_arr(U_snap=usnap, V_snap=vsnap)
plot.pearson_plot(clipped_array=clipped_arr, id_number=2)

plot.animate_snapshots("222735", save_name="Test_miscDots_k0051_F0024.gif")

### Analysis Parameters

Set parameters for the analysis of selected snapshots. Choose between three 
types of blurring: gaussian, median or mecke. 


In [ ]:
analysis_params = au.PipelineConfig(
    input_stack = "",       # gets overwritten in config_dots_stripes()
    folder_name = "",       # same
    blur_type   = "gaussian", # three types: "gaussian", "median", "mecke"
    sigma       = 2.5, # "gaussian"
    ksize       = 3, # "median"
    fwhm_px     = 0.05,    # for "mecke" blur
    gamma       = 2,      # for "mecke" blur
    tag         = None,
)


The configs for dots and stripes are created, with the chosen parameters.

In [ ]:
dots_cfg, stripes_cfg = ph.config_dots_stripes(analysis_params)

### Run analysis pipeline
This runs the full analysis pipeline for both dots and stripes, we always use 
the sigma 0 image_stack as the base, and then apply different blurs if desired.

In [ ]:
dots_results = paths = ph.run_pipeline(
    **dots_cfg.__dict__ 
)

print("\nAll output files:")
for k, v in paths.items():
    print(f"  {k:7s} : {v}")

stripes_results = paths = ph.run_pipeline(
    **stripes_cfg.__dict__ 
)

print("\nAll output files:")
for k, v in paths.items():
    print(f"  {k:7s} : {v}")

### Run pipeline for a single image

This runs the full analysis pipeline for a single image (used for Pearson imgs).

In [ ]:
dots_results = ph.run_pipeline("data/journal/final_presentation/\
                               pearson_converted/NEW_pearson_dots_array.npy",
                folder_name = "pearson/dots/s1",
                blur_type="gaussian", # make sure this is gaussian
                sigma=1, # applying sigma 1 to original image
                levels=256,
                outroot="data/stats/",
                tag=None)
    
stripes_results = ph.run_pipeline("data/journal/final_presentation/\
                                  pearson_converted/NEW_pearson_stripes_array.npy",
                folder_name = "pearson/stripes/s1",
                blur_type="gaussian",
                sigma=1,
                levels=256,
                outroot="data/stats/",
                tag=None)

### Run plotting pipeline

This code runs the entire plotting pipeline for dots vs stripes and plots a 
comparison to Mecke.

In [ ]:
ph.run_dots_vs_stripes_plots(dots_results, stripes_results=stripes_results, n_points=150)

In [ ]:
red   = sns.color_palette("Set1", 4)[3]
green = sns.color_palette("Set1", 4)[2]

dots_stats   = "data/stats/pearson/dots/s1/curves_s1_1runs_stats.npz"
dots_fit     = "data/stats/pearson/dots/s1/curves_s1_1runs_fit.npz"
stripes_stats = "data/stats/pearson/stripes/s1/curves_s1_1runs_stats.npz"
stripes_fit   = "data/stats/pearson/stripes/s1/curves_s1_1runs_fit.npz"
ours_dots_stats = "data/stats/dots/s1/curves_s1_20runs_stats.npz"
ours_dots_fit = "data/stats/dots/s1/curves_s1_20runs_fit.npz"
ours_stripes_stats = "data/stats/stripes/s1/curves_s1_20runs_stats.npz"
ours_stripes_fit = "data/stats/stripes/s1/curves_s1_20runs_fit.npz"

plot_dir = Path("data/journal/final_presentation/comparison_PvsUs_sigma1")
plot_dir.mkdir(parents=True, exist_ok=True)

for metric in ("v", "s", "chi"):
    png = plot_dir/f"pearson_dots_{metric}.png"

    plot.save_raw_data_plots_extended(100, metric, png, dots_stats, dots_fit,
                                    ours_dots_stats, ours_dots_fit,
                                    stripes_stats, stripes_fit, 
                                    ours_stripes_stats, ours_stripes_fit,)


for func in ("pv", "ps", "pchi"):
    png = plot_dir/f"pearson_vs_us_{func}.png"
    plot.plot_functional_comparison(func, png,
                        dots_stats, dots_fit,
                        ours_dots_stats, ours_dots_fit,
                        stripes_stats, stripes_fit, 
                        ours_stripes_stats, ours_stripes_fit,
                        n_points=100)

### Statistical analysis, us vs Pearson
This code was used to generate a statistical comparison between out results and
screenshots from Pearson's simulated results. 

In [15]:
pearson_dots = pfa.load_fit_coeffs("data/stats/pearson/dots/s1/curves_s1_1runs_fit.npz")
pearson_stripes = pfa.load_fit_coeffs("data/stats/pearson/stripes/s1/curves_s1_1runs_fit.npz")
our_dots = pfa.load_fit_coeffs("data/stats/dots/s1/curves_s1_20runs_fit.npz")
our_stripes = pfa.load_fit_coeffs("data/stats/stripes/s1/curves_s1_20runs_fit.npz")

# RMS comparison dots
rms = pfa.compute_rms(pearson_dots["pv"], our_dots["pv"], normalise="range")
print(f"RMS pv between Pearson and Ours (dots): {rms:.4f}")
rms = pfa.compute_rms(pearson_dots["ps"], our_dots["ps"], normalise="range")
print(f"RMS ps between Pearson and Ours (dots): {rms:.4f}")
rms = pfa.compute_rms(pearson_dots["pchi"], our_dots["pchi"], normalise="range")
print(f"RMS pchi between Pearson and Ours (dots): {rms:.4f}")

# RMS comparison stripes 
rms = pfa.compute_rms(pearson_stripes["pv"], our_stripes["pv"], normalise="range")
print(f"RMS pv between Pearson and Ours (stripes): {rms:.4f}")
rms = pfa.compute_rms(pearson_stripes["ps"], our_stripes["ps"], normalise="range")
print(f"RMS ps between Pearson and Ours (stripes): {rms:.4f}")
rms = pfa.compute_rms(pearson_stripes["pchi"], our_stripes["pchi"], normalise="range")
print(f"RMS pchi between Pearson and Ours (stripes): {rms:.4f}")

# Orientation agreement (e.g. do both have similar dots–stripes contrast)
frac, corr = pfa.orientation_agreement(
    our_dots["pv"], our_stripes["pv"],
    pearson_dots["pv"], pearson_stripes["pv"]
)
print(f"Orientation agreement (dots vs stripes, us vs Pearson): same-sign: {frac:.3f}, Pearson corr: {corr:.3f}")

frac, corr = pfa.orientation_agreement(
    our_dots["ps"], our_stripes["ps"],
    pearson_dots["ps"], pearson_stripes["ps"]
)
print(f"Orientation agreement (dots vs stripes, us vs Pearson): same-sign: {frac:.3f}, Pearson corr: {corr:.3f}")

frac, corr = pfa.orientation_agreement(
    our_dots["pchi"], our_stripes["pchi"],
    pearson_dots["pchi"], pearson_stripes["pchi"]
)
print(f"Orientation agreement (dots vs stripes, us vs Pearson): same-sign: {frac:.3f}, Pearson corr: {corr:.3f}")


RMS pv between Pearson and Ours (dots): 0.1306
RMS ps between Pearson and Ours (dots): 0.2827
RMS pchi between Pearson and Ours (dots): 0.2580
RMS pv between Pearson and Ours (stripes): 0.0797
RMS ps between Pearson and Ours (stripes): 0.0831
RMS pchi between Pearson and Ours (stripes): 0.2897
Orientation agreement (dots vs stripes, us vs Pearson): same-sign: 1.000, Pearson corr: 0.851
Orientation agreement (dots vs stripes, us vs Pearson): same-sign: 0.698, Pearson corr: 0.531
Orientation agreement (dots vs stripes, us vs Pearson): same-sign: 1.000, Pearson corr: 0.893
